## 1. Quadratic form positivity

You're presented with the constraint:
\begin{equation}
2x^2 + 2y^2 + 9z^2 + 8xy -6xz - 6yz \le 1 \hspace{2cm} (1)
\end{equation}

**a)** Write the constraint (1) in the standard form $v^\mathsf{T} Q v \le 1$. Where $Q$ is a symmetric matrix. What is $Q$ and what is $v$?	
	
**b)** It turns out the above constraint is _not_ convex. In other words, the set of $(x,y,z)$ satisfying the constraint (1) is not an ellipsoid. Explain why this is the case.

**Note:** you can perform an orthogonal decomposition of a symmetric matrix $Q$ in Julia like this:
```
(L,U) = eig(Q)      # L is the vector of eigenvalues and U is orthogonal
U * diagm(L) * U'   # this is equal to Q (as long as Q was symmetric to begin with)
```

**c)** We can also write the constraint (1) using norms by putting it in the form:
\begin{equation}
\|Av\|^2 - \|Bv\|^2 \le 1
\end{equation}
What is $v$ and what are the matrices $A$ and $B$ that make the constraint above equivalent to (1)?

**d)** Explain how to find a direction for a vector, $(x, y, z)$, such that $\|(x, y, z)\|_2$ is unbounded (can be made arbitrarily large) while satisfying constraint (1).

**Solution to part a):**
We want a matrix, $Q$, that satisfies:

$$\left[\begin{matrix}
    x & y & z
\end{matrix}\right]
\left[\begin{matrix}
    Q_{11} & Q_{12} & Q_{13} \\
    Q_{21} & Q_{22} & Q_{23} \\
    Q_{31} & Q_{32} & Q_{33} \\
\end{matrix}\right]
\left[\begin{matrix}
    x \\ y \\ z
\end{matrix}\right]
\leq 1$$

By inspection, the matrix $Q$ is:
\begin{equation}
Q = \begin{bmatrix} 2 & 4 & -3 \\ 4 & 2 & -3 \\ -3 & -3 & 9 \end{bmatrix}
\end{equation}
and $v$ is the vector $(x,y,z)$.

**Solution to part b)**

We can figure out whether it's an ellipsoid by seeing if the eigenvalues are all nonnegative. Since one of the eigenvalues is negative and the others are positive, Q is indefinite. The constraint is therefore not an ellipsoid. See below.

In [2]:
using LinearAlgebra

Q = [2 4 -3; 4 2 -3; -3 -3 9]
(L, U) = eigen(Q);
L

3-element Vector{Float64}:
 -1.9999999999999996
  3.000000000000001
 12.0

**Solution to part c)**

We can split the constraint into two pieces by separating the positive eigenvalues from the negative ones. Since the eigenvalues are $\{-2,3,12\}$, our decomposition looks like:
\begin{align}
Q &= -2 u_1 u_1^\mathsf{T}+3 u_2 u_2^\mathsf{T}+12 u_3 u_3^\mathsf{T} \\
&= \left(3 u_2 u_2^\mathsf{T}+12 u_3 u_3^\mathsf{T}\right) - \left( 2 u_1 u_1^\mathsf{T} \right) \\
&= \begin{bmatrix}u_2 & u_3\end{bmatrix} \begin{bmatrix}3 & 0 \\ 0 & 12\end{bmatrix} \begin{bmatrix}u_2 & u_3\end{bmatrix}^\mathsf{T} - \begin{bmatrix}u_1\end{bmatrix} \begin{bmatrix}2\end{bmatrix} \begin{bmatrix}u_1\end{bmatrix}^\mathsf{T}
\end{align}
So this looks something like $U_1 Q_1 U_1^\mathsf{T} - U_2 Q_2 U_2^\mathsf{T}$, which is almost what we need. We want this to look like $A^\mathsf{T}A - B^\mathsf{T}B$. So one way to do this is to set:
\begin{equation}
A = \begin{bmatrix}\sqrt3 & 0 \\ 0 & \sqrt{12}\end{bmatrix}\begin{bmatrix}u_2 & u_3\end{bmatrix}^\mathsf{T}
\qquad\text{and}\qquad
B = \sqrt{2}\, u_1^\mathsf{T}
\end{equation}
with $v = (x,y,z)$ as before. Let's test that it gives the correct result for a random point:

In [3]:
using Random

rng = MersenneTwister(1234)  # control our random vector
Q = [2 4 -3; 4 2 -3; -3 -3 9]
(L, U) = eigen(Q);
v = randn(rng, (3, 1));

A = diagm(0 => [sqrt.(3), sqrt.(12)]) * U[:, 2:3]'
B = sqrt.(2) * U[:, 1]'
println("A is equal to: ", A)
println("B is equal to: ", B)
println("v'Qv = ", (v'*Q*v)[1])
println("|Av|^2 - |Bv|^2 = ", norm(A*v)^2 - norm(B*v)^2)

A is equal to: [-1.0 -1.0 -0.9999999999999997; -1.4142135623730943 -1.4142135623730943 2.8284271247461903]
B is equal to: 

[1.0000000000000002 -1.0 0.0]


v'Qv = -1.0276008777535126


|Av|^2 - |Bv|^2 = -1.027600877753513


**Alternative solution to part c)**

We can do a similar split but by keeping the entire matrix together:
\begin{align}
Q &= U \begin{bmatrix}-2 & 0 & 0 \\ 0 & 3 & 0 \\ 0 & 0 & 12 \end{bmatrix} U^\mathsf{T} \\
&= U \begin{bmatrix}0 & 0 & 0 \\ 0 & 3 & 0 \\ 0 & 0 & 12 \end{bmatrix} U^\mathsf{T} - U \begin{bmatrix}2 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \end{bmatrix} U^\mathsf{T}
\end{align}
So this looks something like $U Q_1 U^\mathsf{T} - U Q_2 U^\mathsf{T}$, where the $U$ matrices are square and orthogonal. We can find $A$ and $B$ by doing a matrix square root! This results in:
\begin{equation}
A = U \begin{bmatrix}0 & 0 & 0 \\ 0 & \sqrt{3} & 0 \\ 0 & 0 & \sqrt{12} \end{bmatrix} U^\mathsf{T}
\qquad\text{and}\qquad
B = U \begin{bmatrix}\sqrt{2} & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \end{bmatrix} U^\mathsf{T}
\end{equation}
with $v = (x,y,z)$ as before. Let's test that it gives the correct result for a random point:

In [4]:
A = U * diagm(0 => [0, sqrt.(3), sqrt.(12)]) * U'
B = U * diagm(0 => [sqrt.(2), 0, 0]) * U'
println("A is equal to:", A)
println("B is equal to:", B)
println("v'Qv = ", (v'*Q*v)[1])
println("|Av|^2 - |Bv|^2 = ", norm(A*v)^2 - norm(B*v)^2)

A is equal to:[1.154700538379251 1.154700538379251 -0.5773502691896254; 1.154700538379251 1.154700538379251 -0.5773502691896254; -0.5773502691896254 -0.5773502691896254 2.8867513459481287]
B is equal to:[0.7071067811865477 -0.7071067811865476 0.0; -0.7071067811865476 0.7071067811865475 0.0; 0.0 -0.0 0.0]
v'Qv = -1.0276008777535126
|Av|^2 - |Bv|^2 = -1.027600877753514


**Solution to part d)**

We can make the quadratic form negative. Once we can do that for some $v$, then any scalar multiple of that $v$ will lead to a _more_ negative result, which will still satisfy the constraint. In this case, it amounts to using any $v$ in the directly of $u_1$, since that leads to:
\begin{equation}
u_1^\mathsf{T} Q u_1 = u_1^\mathsf{T} U \begin{bmatrix}-2 & 0 & 0 \\ 0 & 3 & 0 \\ 0 & 0 & 12 \end{bmatrix} U^\mathsf{T} u_1 = -2
\end{equation}
So let's compute $u_1$:

In [5]:
Q = [2 4 -3; 4 2 -3; -3 -3 9]
(L, U) = eigen(Q);

println("The following (x,y,z) makes the quadratic form negative:")
vbad = U[:, 1]
# display(vbad)
vbad = [-1,1,0]

# let's verify that it actually makes the quadratic form negative. We can take any multiple of this
println("Value of quadratic form for this v")
sleep(.1)
(x, y, z) = vbad
display(2x^2 + 2y^2 + 9z^2 + 8x*y -6x*z - 6y*z)

The following (x,y,z) makes the quadratic form negative:
Value of quadratic form for this v


-4

Now we have this value $v_\text{bad} = u_1$ with norm $1$ that leads to $v_\text{bad}^\mathsf{T}Qv_\text{bad} = -2 \le 1$. Therefore, if we scale it by some $t > 0$, we get a norm of $t$ and $(v_\text{bad}t)^\mathsf{T}Q(v_\text{bad}t) = -2t^2 \le 1$. So we can make this vector as large as we want in norm by scaling it and it will still satisfy the constraint.

## 2. Enclosing circle

Given a set of points in the plane $x_i \in \mathbb{R}^2$, we would like to find the circle
with smallest possible area that contains all of the points. Explain how to model this as an optimization
problem. To test your model, generate a set of 50 random points using the code `X = 4.+randn(2,50)`
(this generates a $2\times 50$ matrix $X$ whose columns are the $x_i$). Produce a plot of the randomly generated
points along with the enclosing circle of smallest area.

### Model
Input: Set of 50 random points X. ($X_{k,1}$,$X_{k,2}$) is the kth point's position.

Variables: 

xc1,xc2 (the position of the circle's center)

r>=0 (the radius)

Model:

$$\begin{aligned}
\text{minimize}\qquad& r \\
\text{subject to:}\qquad& (X_{k,1}-xc1)^2 + (X_{k,2}-xc2)^2 \leq r^2 \qquad \forall k = 1,2,\cdots,50  \qquad \text{every point is contained in the circle}\\
& r \ge 0 
\end{aligned}$$

In [6]:
N = 50               # number of points
X = 4 .+ randn(2,N)   # generate random points

# plot 
t = range(0,stop=2π,length=100)   # parameter that traverses the circle

# solve the problem --- this is an SOCP.
using JuMP, Gurobi, LinearAlgebra

m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))

@variable(m, xc[1:2]) # center of circle
@variable(m, r) # radius square

@constraint(m, constr[i in 1:N], [r; X[:,i] - xc] in SecondOrderCone()) # every point is in the circle
@objective(m, Min, r)

optimize!(m);

# plot the points and the circle
c1 = JuMP.value(xc[1])
c2 = JuMP.value(xc[2])
ropt = JuMP.value(r)

println("The center of the circle is (",c1,", ", c2, "). Radius is ", ropt)

using PyPlot
t = range(0,stop=2π,length=100)
axis("equal")
scatter(c1, c2, color = "red");
scatter( X[1,:], X[2,:], color = "black")
plot( c1 .+ ropt*cos.(t), c2 .+ ropt*sin.(t))


Academic license - for non-commercial use only - expires 2022-08-07
The center of the circle is (3.47950075794807, 3.949190917763403). Radius is 3.054419654354594


1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x000000009C158280>

## 3. Huber loss

In statistics, we frequently encounter data sets containing  _outliers_, which are bad data points arising from experimental error or abnormally high noise. Consider for example the following data set consisting of 15 pairs $(x,y)$.

|$x$ | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|$y$ | 6.31 | 3.78 | 5.12 | 1.71 | 2.99 | 4.53 | 2.11 | 3.88 | 4.67 | 26 | 2.06 | 23 | 1.58 | 2.17 | 0.02|

The $y$ values corresponding to $x=10$ and $x=12$ are outliers because they are far outside the expected range of values for the experiment.

**(a)** Compute the best linear fit to the data using an $\ell_2$ cost (least squares). In other words, we are looking for the $a$ and $b$ that minimize the expression:
$$
\text{$\ell_2$ cost:}\qquad 
\sum_{i=1}^{15} (y_i - a x_i - b)^2
$$
Repeat the linear fit computation but this time exclude the outliers from your data set. On a single plot, show the data points and both linear fits. Explain the difference between both fits.

**solution**
The one without outliers has lower slope than the one with outliers. And it is obvious that the one without outliers fits better to the dataset without outliers.


**(b)** It's not always practical to remove outliers from the data manually, so we'll investigate ways of automatically dealing with outliers by changing our cost function. Find the best linear fit again (including the outliers), but this time use the $\ell_1$ cost function:
$$
\text{$\ell_1$ cost:}\qquad 
\sum_{i=1}^{15} \left|\,y_i - a x_i - b\,\right|
$$
Include a plot containing the data and the best $\ell_1$ linear fit. Does the $\ell_1$ cost handle outliers better or worse than least squares? Explain why.
%You should notice that the $\ell_1$ cost handles outliers better than least squares but there is still a bias; there are many more points below the line than above.

**solution**
The $\ell_1$ handles the outliers better since the same residual will be larger in $\ell_2$ than that in $\ell_1$.


**(c)** Another approach is to use an $\ell_2$ penalty for points that are close to the line but an $\ell_1$ penalty for points that are far away. Specifically, we'll use something called the _Huber loss_, defined as:
$$
\phi(x) = \begin{cases}
x^2 & \text{if } -M\le x \le M \\
2M|x|-M^2 & \text{otherwise}
\end{cases}
$$
Here, $M$ is a parameter that determines where the quadratic function transitions to a linear function. The formula above is simple, but not in a form that is useful for us. As it turns out, we can evaluate the Huber loss function at any point $x$ by solving the following convex QP instead:
$$
\phi(x) = \left\{ \begin{aligned}
\underset{v,w}{\text{minimize}}\quad & w^2 + 2 M v \\
\text{subject to:}\quad & |x| \le w + v \\
&v \ge 0,\,\, w \le M
\end{aligned}\right\}
$$
Verify this fact by solving the above QP (with $M=1$) for many values of $x$ in the interval $-3 \le x \le 3$ and reproducing the plot above. Finally, find the best linear fit to our data  using a Huber loss with $M=1$ and produce a plot showing your fit. The cost function is:
$$
\text{Huber loss:}\qquad 
\sum_{i=1}^{15} \phi(\,y_i - a x_i - b)
$$

### part a model

Input: data points x, y (To exclude the outliers, just remove the 10th and 12th entry from x and y)

Variables:
a, b (The coefficients in the l2 cost)


Model:
$$\begin{aligned}
\text{minimize}\qquad& \sum_{i=1}^{15} (y_i - ax_i -b)^2 \\
\end{aligned}$$


### part b model

Input: data points x, y (To exclude the outliers, just remove the 10th and 12th entry from x and y)

Variables:
a, b (The coefficients in the l2 cost)
$t_i~i=1,2,\cdots,15$ (Variables for the absolute value)


Model:
$$\begin{aligned}
\text{minimize}\qquad& \sum_{i=1}^{15} t_i \\
& y_i - ax_i -b \leq t_i \qquad \forall i \\
& -y_i + ax_i +b \leq t_i \qquad \forall i \\
\end{aligned}$$


### part c model

Input: data points x, y (To exclude the outliers, just remove the 10th and 12th entry from x and y)

Variables:
a, b (The coefficients in the l2 cost)
$w_i~i=1,2,\cdots,15$ (Variables required by the Huber loss function)
$v_i~i=1,2,\cdots,15$ (Variables required by the Huber loss function)

Model:
$$\begin{aligned}
\text{minimize}\qquad& \sum_{i=1}^{15} w_i^2 + 2Mv_i \qquad \text{Sum of the Huber loss for every data point} \\
& y_i - ax_i -b \leq w_i + v_i \qquad \forall i \\
& -y_i + ax_i +b \leq w_i + v_i \qquad \forall i \\
& w_i \leq M \qquad \forall i \\
\end{aligned}$$

In [7]:
using JuMP,Gurobi

y = [6.31, 3.78, 5.12, 1.71, 2.99, 4.53, 2.11, 3.88, 4.67, 26, 2.06, 23, 1.58, 2.17, 0.02]
x = 1:15

# PART A code

# find the least-squares fit using ALL the data points
c = 1:15
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
@variable(m, a)
@variable(m, b)
@expression(m, t, sum((y[i] - a*x[i] - b)^2 for i in c ))
@objective(m, Min, t)
optimize!(m)
status = termination_status(m)
a1,b1 = JuMP.value(a),JuMP.value(b)

# find the least squares fit REMOVING OUTLIERS
c = [1:9; 11:11; 13:15]
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
@variable(m, a)
@variable(m, b)
@expression(m, t, sum((y[i] - a*x[i] - b)^2 for i in c ))
@objective(m, Min, t)
optimize!(m)
status = termination_status(m)
a2,b2 = JuMP.value(a),JuMP.value(b)

# PART B code

# find the L1 fit using ALL the data points
c = 1:15
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
@variable(m, a)
@variable(m, b)
@variable(m, t[1:15] >= 0)
@constraint(m, flow1[i in c], y[i] - a*x[i] - b <=  t[i])
@constraint(m, flow2[i in c], y[i] - a*x[i] - b >= -t[i])
@objective(m, Min, sum(t))
optimize!(m)
status = termination_status(m)
a3,b3 = JuMP.value(a),JuMP.value(b)

# find the L1 fit REMOVING OUTLIERS
c = [1:9; 11:11; 13:15]
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
@variable(m, a)
@variable(m, b)
@variable(m, t[1:15] >= 0)
@constraint(m, flow1[i in c], y[i] - a*x[i] - b <=  t[i])
@constraint(m, flow2[i in c], y[i] - a*x[i] - b >= -t[i])
@objective(m, Min, sum(t))
optimize!(m)
status = termination_status(m)
a4,b4 = JuMP.value(a),JuMP.value(b)
;

Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07


In [8]:
# PART C (first part)

M = 1
hx = range(-3,stop=3,length=50)

# compute Huber loss using formula
hy1 = zeros(size(hx))
for (i,xx) in enumerate(hx)
    if abs(xx) <= M
        hy1[i] = xx^2
    else
        hy1[i] = 2*M*abs(xx) - M^2
    end
end

# compute Huber loss using QP
hy2 = zeros(size(hx))
for (i,xx) in enumerate(hx)
    m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
    @variable(m, v >= 0)
    @variable(m, w <= M)
    @constraint(m, abs(xx) <= v + w)
    @objective(m, Min, w^2 + 2*M*v)
    optimize!(m)
    hy2[i] = JuMP.objective_value(m)
end

using PyPlot
figure(figsize = (12,3))

subplot(1,2,1)
plot(hx,hy1)
axis([-3,3,0,5])
title("Plot of Huber loss using formula")

subplot(1,2,2)
plot(hx,hy2)
axis([-3,3,0,5])
title("Plot of Huber loss using QP")
;

Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only -

In [9]:
# PART C code

# find the Huber fit using ALL the data points
c = 1:15
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
@variable(m, a)
@variable(m, b)
@variable(m, v[1:15] >= 0)
@variable(m, w[1:15] <= M)
@constraint(m, flow1[i in c], y[i] - a*x[i] - b <=  w[i] + v[i])
@constraint(m, flow2[i in c], y[i] - a*x[i] - b >= -w[i] - v[i])
@objective(m, Min, sum(w.^2) + 2*M*sum(v))
optimize!(m)
status = termination_status(m)
a5,b5 = JuMP.value(a),JuMP.value(b)

# find the Huber fit REMOVING OUTLIERS
c = [1:9;11:11; 13:15]
m = Model(optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0))
@variable(m, a)
@variable(m, b)
@variable(m, v[1:15] >= 0)
@variable(m, w[1:15] <= M)
@constraint(m, flow1[i in c], y[i] - a*x[i] - b <=  w[i] + v[i])
@constraint(m, flow2[i in c], y[i] - a*x[i] - b >= -w[i] - v[i])
@objective(m, Min, sum(w.^2) + 2*M*sum(v))
optimize!(m)
status = termination_status(m)
a6,b6 = JuMP.value(a),JuMP.value(b)
;

Academic license - for non-commercial use only - expires 2022-08-07
Academic license - for non-commercial use only - expires 2022-08-07


In [10]:
# Make all the plots

using PyPlot
figure(figsize = (17,7))
scatter(x,y,label="data",color = "black")
scatter([x[10],x[12]],[y[10],y[12]],label="outliers",color = "red") # outliers
plot(x, a1*x .+ b1,label="l2 with outliers")
plot(x, a3*x .+ b3,label="l1 with outliers")
plot(x, a5*x .+ b5,label="Huber with outliers")
plot(x, a2*x .+ b2,label="l2 without outliers")
plot(x, a4*x .+ b4,label="l1 without outliers")
plot(x, a6*x .+ b6,label="Huber without outliers")
legend(loc ="best")
ylabel("y")
xlabel("x")
;